## GZ2フルデータセットからML対象データセットを作成（ランダムサンプリング・データセット）

In [1]:
import os, shutil
import sys, time
import numpy as np
import pandas as pd
from astropy.io import fits

# 作業用ディレクトリ定義（要カスタマイズ）
galaxy_data_directory = '/home/satoshi/Galaxy/'

# 新カテゴリカタログ
gz2_catalog_v2 = galaxy_data_directory + 'gz2_catalog_hubble_12class.fits'

# データセットベースディレクトリ
dataset_directory_base = galaxy_data_directory + 'dataset_12class_full_256_01'
# サンプル件数/クラス
sample_size = 4000

sample_directory_base = galaxy_data_directory + 'dataset_12class_random_256_01_' + str(sample_size)
os.mkdir(sample_directory_base)


In [2]:
with fits.open(gz2_catalog_v2) as hdul:
    data = hdul[1].data
    
    print('category_catalog_hubble rows = ', len(data))
    galaxy_class = np.array(data.field('HUBBLE_CLASS'))
    # FITSのBig endianをPandasのLittle endianに変換
    #galaxy_dr7objid = np.array(data.field('dr7objid').byteswap().newbyteorder())
    galaxy_redshift = np.array(data.field('REDSHIFT_SIMPLE_BIN').byteswap().newbyteorder())
    galaxy_mgr = np.array(data.field('PETROMAG_MR_SIMPLE_BIN').byteswap().newbyteorder())
    galaxy_fname = np.array(data.field('FILE_NAME'))
    

category_catalog_hubble rows =  239100


In [3]:
galaxy_class_series = pd.Series(galaxy_class)
galaxy_fname_series = pd.Series(galaxy_fname)
galaxy_redshift_series = pd.Series(galaxy_redshift)
galaxy_mgr_series = pd.Series(galaxy_mgr)
#galaxy_dr7objid_series = pd.Series(galaxy_dr7objid)
galaxy_dataframe = pd.DataFrame({'galaxy_class' : galaxy_class_series,
                                                             'galaxy_fname' : galaxy_fname_series,
                                                             'galaxy_redshift' : galaxy_redshift_series, 
                                                             'galaxy_mgr' : galaxy_mgr_series})

#print(galaxy_dataframe)

unique_galaxy_class = galaxy_dataframe['galaxy_class'].unique()
print(unique_galaxy_class, '  ', len(unique_galaxy_class))

for class_name in unique_galaxy_class :
    os.mkdir(sample_directory_base + '/' + class_name)
    

['Sa' 'E3' 'E0' 'Sb' 'SBb' 'S0' 'E7' 'SB0' 'Edgeon' 'SBa' 'SBc' 'Sc']    12


#### データをランダムにサンプル件数分サンプリングする

In [4]:
def select_random(df):
    return df.sample(n=sample_size)

random_galaxy = galaxy_dataframe.groupby('galaxy_class').apply(select_random)

for galaxy in random_galaxy.values:
    
    class_name = galaxy[0]
    fname = galaxy[1]
    #print(fname)
    src = os.path.join(dataset_directory_base + '/' + class_name, fname)
    dst = os.path.join(sample_directory_base + '/' + class_name, fname)
    shutil.copyfile(src, dst)

print('completed')

completed
